In [ ]:
import pandas as pd
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from timeit import default_timer as timer
import matplotlib
from matplotlib.legend_handler import HandlerLine2D

In [ ]:
mb_keys_dict = {
    'crop': np.array([18,19,39,20,40,62,41,36,46,47,35,48]),
    'forest': np.array([3]),
    'savanna': np.array([4]),
    'grassland':np.array([12]),
    'pasture': np.array([15])
}

In [ ]:
def read_process_lulc_csv(csv):
    df = pd.read_csv(csv, index_col=0)
    df = df.loc[:, df.columns[-1]]
    lulc_y = int(df.name[42:46]) # For all of mato grosso

    df = pd.DataFrame(list(df.apply(eval).values), index=df.index)
    df.columns = pd.MultiIndex.from_product([[lulc_y], df.columns])
    return df

def summarize_lulc(year_df):
    out_df = pd.DataFrame()
    for lulc_class in mb_keys_dict.keys():
        sum_of_class = year_df.loc[:, np.in1d(year_df.columns, mb_keys_dict[lulc_class])].sum(axis=1)
        out_df[lulc_class] = sum_of_class
    return out_df

def classify_lulc(lulc_counts_df):

    natural_covers = ['forest', 'grassland', 'savanna']
    ag_covers = ['crop','pasture']
    natural_df = lulc_counts_df[natural_covers]
    ag_df = lulc_counts_df[ag_covers]
    ag_sum = ag_df.sum(axis=1)
    natural_sum = natural_df.sum(axis=1)
    is_natural = ((ag_sum < 1000) & (natural_sum> 1000))
    natural_cover = natural_df.loc[is_natural].idxmax(axis=1)
    is_ag = ((~is_natural) & (ag_sum > 1000))
    ag_cover = ag_df.loc[is_ag].idxmax(axis=1)
    lulc_counts_df['class'] = 'other'
    lulc_counts_df.loc[is_ag, 'class'] = ag_cover
    lulc_counts_df.loc[is_natural, 'class'] = natural_cover
    return lulc_counts_df['class']

def calc_lulc_full(df):
    new_df = pd.DataFrame(df.index).set_index('id_in_tile')

    for y in df.columns.levels[0]:
        year_df = summarize_lulc(df[y])
        year_df.index = df.index
        classes = classify_lulc(year_df)
        new_df.loc[:, y] = classes
        
    return new_df

def process_year_lulc(y):
    all_csvs = glob.glob('./out/lulc_res_counts/lulc_stats_res_{}_mb_*_counts.csv'.format(y))
    all_csvs.sort()
    areas = pd.read_csv(all_csvs[0])['area'].values
    full_df = pd.concat([read_process_lulc_csv(csv) for csv in all_csvs], axis=1)
    lulc_df = calc_lulc_full(full_df)
    lulc_df['area'] = areas
    return lulc_df

In [ ]:
year_range = np.arange(1985, 2022)

for y in year_range:
    lulc_y_csv = './out/res_lulc_processed/res_lulc_{}.csv'.format(y)
    if not os.path.isfile(lulc_y_csv):
        lulc_y_df = process_year_lulc(y)
        lulc_y_df.to_csv('./out/res_lulc_processed/res_lulc_{}.csv'.format(y))
    print(y)


# Helper functions/data for calculating years since/before transitions

In [ ]:
lulc_transitions_area_df = pd.read_csv('./out/lulc_summaries/lulc_transitions_yearof_only.csv')
lulc_transitions_area_df['year'] = lulc_transitions_area_df['year'].astype(int)
lulc_transitions_area_df = lulc_transitions_area_df.drop(columns=['Unnamed: 0']).set_index('year')
# Convert areas to km2
lulc_transitions_area_df = lulc_transitions_area_df * 30 * 30 / 1000000

In [ ]:

def get_transition_counts(lulc_df, res_y, 
                          only_to_next_transition=True, only_permanent_transition=False,
                          trans_area_df=None, adjust_for_lulc_area=True):
    transition_counts = []
    for y in range(1985, 2022):
        transition_df = lulc_df.loc[lulc_df[y]!=lulc_df[y+1]]
        if only_permanent_transition:
            # only count transitions that match current type and remained that way all along
            if res_y <= y:
                # Transition is in the future
                transition_df = transition_df.loc[
                    np.all(transition_df.loc[:, res_y:y].eq(transition_df[res_y].values, axis=0), axis=1)
                    ]
            if res_y > y:
                # Transition is in the past
                transition_df = transition_df.loc[
                    np.all(transition_df.loc[:, y+1:res_y].eq(transition_df[res_y].values, axis=0), axis=1)
                ]
        elif only_to_next_transition:
            # only count transitions that match current type
            if res_y <= y:
                # Transition is in the future
                transition_df = transition_df.loc[transition_df[y]==transition_df[res_y]]
            if res_y > y:
                # Transition is in the past
                transition_df = transition_df.loc[transition_df[y+1]==transition_df[res_y]]
        
        transition_df['trans_type'] = transition_df[y] + '-' + transition_df[y+1]
        trans_summaries = transition_df[['area','trans_type']].groupby('trans_type').aggregate(
            ['count', 'sum'])
        trans_summaries.name = y
        if adjust_for_lulc_area:
            # Fill in 0s for no reservoir transitions
            missing_transitions = trans_area_df.columns[~trans_area_df.columns.isin(trans_summaries.index)]
            for missing in missing_transitions:
                trans_summaries.loc[missing] = 0
            trans_summaries = trans_summaries.loc[trans_area_df.columns]
            trans_summaries.columns = trans_summaries.columns.set_levels([y],level=0)
            trans_summaries[(y, 'count')] = trans_summaries[(y,'count')]/trans_area_df.loc[y]
            trans_summaries[(y,'sum')] = trans_summaries[(y,'sum')]/trans_area_df.loc[y]
            trans_summaries.columns = trans_summaries.columns.reorder_levels([1,0])
        transition_counts.append(trans_summaries)
    full_year_transitions = pd.concat(transition_counts, axis=1)
    return full_year_transitions

# First for all 7 types of transitions

In [ ]:
normalized_dfs_sum = []
normalized_dfs_count = []
for res_y in year_range:
    csv = './out/res_lulc_processed/res_lulc_{}.csv'.format(res_y)
    lulc_df = pd.read_csv(csv).set_index('id_in_tile')
    lulc_df.columns = list(lulc_df.columns[:-1].astype(int)) + [lulc_df.columns[-1]]
    transition_df = get_transition_counts(lulc_df, res_y, only_to_next_transition=True, only_permanent_transition=True,
                                          trans_area_df=lulc_transitions_area_df, adjust_for_lulc_area=True)
    trans_df_count = transition_df['count'].T
    trans_df_count.index = (res_y+1) - trans_df_count.index
    trans_df_sum = transition_df['sum'].T
    trans_df_sum.index = (res_y+1) - trans_df_sum.index
    trans_df_sum.name=res_y
    trans_df_count.name=res_y
    normalized_dfs_sum.append(trans_df_sum)
    normalized_dfs_count.append(trans_df_count)

In [ ]:
full_normalized_df = pd.concat(normalized_dfs_count)
mean_trajectories = full_normalized_df.groupby(level=0).mean()
std_trajectories = full_normalized_df.groupby(level=0).std()
count_trajectories = full_normalized_df.groupby(level=0).count()
ci_low_df = count_trajectories.copy()
ci_high_df = count_trajectories.copy()

In [ ]:
def calc_interval_low(a):
    return st.t.interval(0.95, len(a)-1, loc=np.mean(a), scale=st.sem(a))[0]
def calc_interval_high(a):
    return st.t.interval(0.95, len(a)-1, loc=np.mean(a), scale=st.sem(a))[1]
for trans_type in ci_low_df.columns:
    ci_low_df[trans_type] = full_normalized_df[trans_type].groupby(level=0).apply(calc_interval_low)
    ci_high_df[trans_type] = full_normalized_df[trans_type].groupby(level=0).apply(calc_interval_high)

In [ ]:

rename_dict = {
        'forest-pasture': 'Forest-Pasture',
        'grassland-pasture':'Grassland-Pasture',
        'savanna-pasture':'Savanna-Pasture',
        'forest-crop': 'Forest-Crop',
        'grassland-crop':'Grassland-Crop',
        'savanna-crop':'Savanna-Crop',
        'pasture-crop':'Pasture-Crop',
    }

In [ ]:
mean_trajectories = mean_trajectories.rename(
    columns=rename_dict
)
std_trajectories = std_trajectories.rename(
    columns=rename_dict
)
ci_low_df = ci_low_df.rename(
    columns=rename_dict
)
ci_high_df = ci_high_df.rename(
    columns=rename_dict
)

In [ ]:
# top_traj = mean_trajectories + std_trajectories
# bottom_traj = mean_trajectories - std_trajectories
top_traj = ci_high_df
bottom_traj = ci_low_df

In [ ]:
ax = lulc_transitions_area_df.rename(columns=rename_dict).plot()
ax.set_xlabel('Year')
ax.set_ylabel('Transition Area ($km^2$)')

In [ ]:

# Together
fig, axs = plt.subplots(1,2, figsize=(12,6))
mean_traj_toplot = mean_trajectories[['Forest-Pasture','Grassland-Pasture', 'Savanna-Pasture']]
mean_traj_toplot.plot(xlim=[-25, 25],ylim=[0,0.35], ax=axs[0])
for cover_type in mean_traj_toplot.columns:
    axs[0].fill_between(mean_traj_toplot.index,
                        bottom_traj[cover_type], top_traj[cover_type], alpha=0.15)
mean_traj_toplot = mean_trajectories[['Forest-Crop','Savanna-Crop',  'Grassland-Crop', 'Pasture-Crop']]
mean_traj_toplot.plot(xlim=[-25, 25], ylim=[0,0.35], ax=axs[1])
for cover_type in mean_traj_toplot.columns:
    axs[1].fill_between(mean_traj_toplot.index,
                        bottom_traj[cover_type], top_traj[cover_type], alpha=0.15)
for i, ax in enumerate(axs):
    ax.set_xlabel('Year')
    ax.set_ylabel('Reservoir Density (count per $km^2$)')
    ax.axvline(0,color='darkred', linestyle='--')
fig.tight_layout()
plt.show()

# Simplify: Nature, Pasture, Crop

In [ ]:
transition_regroup_dict = {
    'Natural-Pasture': ['forest-pasture', 'savanna-pasture','grassland-pasture'],
    'Natural-Crop': ['forest-crop', 'savanna-crop','grassland-crop'],
    'Pasture-Crop': ['pasture-crop'],
}

lulc_transitions_area_simple_df = pd.DataFrame({'year':lulc_transitions_area_df.index}).set_index('year')
for key, val in transition_regroup_dict.items():
    lulc_transitions_area_simple_df.loc[:, key] = lulc_transitions_area_df[val].sum(axis=1)

In [ ]:
# Group trajectories into nature, crop, pasture
regroup_dict = {
    'pasture': 'Pasture',
    'forest': 'Natural',
    'savanna': 'Natural',
    'grassland': 'Natural',
    'crop': 'Crop'
}

normalized_dfs_simple_sum = []
normalized_dfs_simple_count = []
for res_y in year_range:
    csv = './out/res_lulc_processed/res_lulc_{}.csv'.format(res_y)
    lulc_df = pd.read_csv(csv).set_index('id_in_tile').replace(regroup_dict)
    lulc_df.columns = list(lulc_df.columns[:-1].astype(int)) + [lulc_df.columns[-1]]
    transition_df = get_transition_counts(lulc_df, res_y, only_to_next_transition=True,only_permanent_transition=True,
                                          trans_area_df=lulc_transitions_area_simple_df, adjust_for_lulc_area=True)
    trans_df_count = transition_df['count'].T
    trans_df_count.index = (res_y+1) - trans_df_count.index
    trans_df_sum = transition_df['sum'].T
    trans_df_sum.index = (res_y+1) - trans_df_sum.index
    trans_df_sum.name=res_y
    trans_df_count.name=res_y
    normalized_dfs_simple_sum.append(trans_df_sum)
    normalized_dfs_simple_count.append(trans_df_count)


In [ ]:
simple_full_normalized_df = pd.concat(normalized_dfs_simple_count)
simple_mean_trajectories = simple_full_normalized_df.groupby(level=0).mean()
simple_std_trajectories = simple_full_normalized_df.groupby(level=0).std()
simple_count_trajectories = simple_full_normalized_df.groupby(level=0).count()
simple_ci_low_df = simple_count_trajectories.copy()
simple_ci_high_df = simple_count_trajectories.copy()

In [ ]:
for trans_type in simple_ci_low_df.columns:
    simple_ci_low_df[trans_type] = simple_full_normalized_df[trans_type].groupby(level=0).apply(calc_interval_low)
    simple_ci_high_df[trans_type] = simple_full_normalized_df[trans_type].groupby(level=0).apply(calc_interval_high)

In [ ]:
fig = plt.figure(figsize=(15, 5))
ax = simple_mean_trajectories.plot(xlim=[-25, 25],ylim=[0,0.30])
for cover_type in simple_mean_trajectories.columns:
    ax.fill_between(simple_mean_trajectories.index,
                    (simple_ci_low_df)[cover_type],
                    (simple_ci_high_df)[cover_type],
                    alpha=0.15)
# colors=['C0','C1','C2']
# for i,cover_type in enumerate(simple_mean_trajectories.columns):
#     ax.scatter(simple_full_normalized_df.index, simple_full_normalized_df[cover_type],
#                alpha=0.3,color=colors[i], edgecolors='none',s=18)
ax.set_xlabel('Year')
ax.set_ylabel('Reservoir Density (count per $km^2$)')
ax.axvline(0,color='darkred', linestyle='--')
fig.show()

In [ ]:
ax = lulc_transitions_area_simple_df.plot()
ax.set_xlabel('Year')
ax.set_ylabel('Transition Area ($km^2$)')

# General LULC fig

In [ ]:
annual_lulc = pd.read_csv('./csvs/lulc_summary_mt.csv').set_index('year')
annual_lulc*= (900/1000000)
annual_lulc['Total Agricultural'] = annual_lulc['pasture'] +  annual_lulc['crop']
annual_lulc.rename(columns={'pasture': 'Pasture', 'crop': 'Crop'}, inplace=True)

In [ ]:
yearof_res_list = []
for res_y in range(1985, 2022):
    yearof_res_list.append(pd.read_csv('./out/res_lulc_processed/res_lulc_{}.csv'.format(res_y))[str(res_y)])

In [ ]:
yearof_counts_by_lulc = [df.value_counts() for df in yearof_res_list]
annual_res_lulc = pd.concat(yearof_counts_by_lulc, axis=1).T

In [ ]:

annual_res_lulc = annual_res_lulc#.drop(columns=['other', 'forest'])
annual_res_lulc = annual_res_lulc.rename(columns={'crop': 'Crop','pasture':'Pasture'})
annual_res_lulc['Total Agricultural'] = annual_res_lulc['Crop'] + annual_res_lulc['Pasture']
annual_res_lulc.columns.name = None
annual_res_lulc_density = annual_res_lulc[['Total Agricultural', 'Pasture', 'Crop']]/annual_lulc[['Total Agricultural', 'Pasture', 'Crop']].loc[:2021].values

In [ ]:

# Together
fig, axs = plt.subplots(1,3, figsize=(15,5))
annual_res_lulc[['Total Agricultural', 'Pasture', 'Crop']].plot(ax=axs[1])
annual_lulc[['Total Agricultural', 'Pasture', 'Crop']].plot(ax=axs[0])
annual_res_lulc_density[['Total Agricultural', 'Pasture', 'Crop']].plot(ax=axs[2])
axs[0].set_xlabel('Year')
axs[1].set_xlabel('Year')
axs[2].set_xlabel('Year')
axs[0].set_ylabel('Area ($km^2$)')
axs[1].set_ylabel('Reservoir Count')
axs[2].set_ylabel('Reservoir Density (count per $km^2$)')
axs[0].set_title('Total Agricultural Area')
axs[1].set_title('Reservoir Count by LULC Class')
axs[2].set_title('Reservoir Density by LULC Class')
fig.tight_layout()
plt.show()

# Single year plot

In [ ]:
# First some quick calcs used for the methods discussion
# y=2000
# print(lulc_transitions_area_simple_df.loc[y])
# print(simple_full_normalized_df_wyears.loc[
#     simple_full_normalized_df_wyears['trans_year']==y].drop(
#         columns=['trans_year'])['Natural-Pasture']*lulc_transitions_area_simple_df.loc[y, 'Natural-Pasture'])
# print(simple_full_normalized_df_wyears.loc[
#     simple_full_normalized_df_wyears['trans_year']==y].drop(
#         columns=['trans_year'])['Natural-Pasture'])

In [ ]:
target_years = np.arange(1990, 2021, 5)
print(target_years)

In [ ]:
simple_full_normalized_df_wyears = simple_full_normalized_df.copy()
simple_full_normalized_df_wyears['res_year'] = np.repeat(year_range, year_range.shape[0])
simple_full_normalized_df_wyears['trans_year'] = (simple_full_normalized_df_wyears['res_year']-simple_full_normalized_df_wyears.index)+1

In [ ]:

def update_prop(handle, orig):
    handle.update_from(orig)
    x,y = handle.get_data()
    handle.set_data([np.mean(x)]*2, [0, 2*y[0]])

In [ ]:
fig, axs = plt.subplots(4, 2, figsize=(15, 15))
for i, ax in enumerate(axs.flatten()):
    if i < (len(target_years)):
        y = target_years[i]
        simple_full_normalized_df_wyears.loc[
            simple_full_normalized_df_wyears['trans_year']==y].drop(
                columns=['trans_year']).plot(ax=ax, x='res_year', legend=False)
        ax.axvline(y,color='darkred', linestyle='--')
        ax.set_xlabel('Year')
        ax.set_ylabel('Reservoir Density (count per $km^2$)')
        ax.set_title(y) 
        prev_ax = ax
    # Add legend
    else:
        ax.set_axis_off()
        handles, labels = prev_ax.get_legend_handles_labels()
        vert_line = matplotlib.lines.Line2D(
            [0, 1],[1,1], color='darkred',lw=1.2, linestyle='--')
        handles.append(vert_line)
            
        # handles.append(matplotlib.lines.Line2D(
        #     [0, 1],[0.5,0.5], color='darkred',lw=1, linestyle='None', marker='|', markersize=10, markeredgewidth=1.2)
        #     )

        labels.append('Transition Year')

        ax.legend(handles, labels,loc=10,
                   fontsize=16, markerscale=2,
                   handler_map={vert_line:HandlerLine2D(update_func=update_prop)}) 

fig.tight_layout()

# Generalized Additive Model

In [ ]:
import pygam

In [ ]:
gam_df = pd.DataFrame(columns=['years','trans_type','density'])

In [ ]:
gam_df['density'] = simple_full_normalized_df.values.flatten()

In [ ]:
gam_df['trans_type'] = np.tile(simple_full_normalized_df.columns.values, simple_full_normalized_df.shape[0])
gam_df['res_year'] = np.repeat(year_range, year_range.shape[0]*simple_full_normalized_df.shape[1])
gam_df['years'] = np.repeat(simple_full_normalized_df.index.values, simple_full_normalized_df.shape[1])
gam_df['trans_year'] = (gam_df['res_year']-gam_df['years'])+1

In [ ]:
cover_type_dict = {
    'Natural-Pasture':0,
    'Natural-Crop':1,
    'Pasture-Crop':2
}

In [ ]:
gam_df['trans_type_int'] = gam_df['trans_type'].replace(cover_type_dict)

In [ ]:
y = gam_df['density'].values
x = gam_df[['years','trans_year','trans_type_int']].values

In [ ]:
lams = np.random.rand(100,3)
lams = np.exp(lams)
gam = pygam.LinearGAM(pygam.s(0) + pygam.s(1) + pygam.f(2)).gridsearch(x, y,lam=lams)
gam.summary()

In [ ]:
for i, term in enumerate(gam.terms):
    if term.isintercept:
        continue

    XX = gam.generate_X_grid(term=i)
    pdep, confi = gam.partial_dependence(term=i, X=XX, width=0.95)

    plt.figure()
    plt.plot(XX[:, term.feature], pdep)
    plt.plot(XX[:, term.feature], confi, c='r', ls='--')
    plt.title(repr(term))
    plt.show()


In [ ]:
# 3D Plot

# import matplotlib.pyplot as plt
# from mpl_toolkits import mplot3d
# plt.ion()
# plt.rcParams['figure.figsize'] = (12, 8)

# XX = gam.generate_X_grid(term=3, meshgrid=True)
# Z = gam.partial_dependence(term=3, X=XX, meshgrid=True)

# ax = plt.axes(projection='3d')
# ax.plot_surface(XX[0], XX[1], Z, cmap='viridis')



### GAM: Fit separate models

In [ ]:
for cover_type in ['Natural-Pasture','Natural-Crop','Pasture-Crop']:
    temp_df = gam_df.loc[gam_df.trans_type==cover_type]
    y = temp_df['density'].values
    x = temp_df[['years', 'trans_year']].values
    # x = np.array([temp_df['years'].values]).T

    # lams = np.random.rand(100,1)
    # lams = np.exp(lams)
    # single_gam = pygam.LinearGAM(pygam.s(0)).gridsearch(x, y,lam=lams, progress=False)
    lams = np.random.rand(100,2)
    lams = np.exp(lams)
    single_gam = pygam.LinearGAM(pygam.s(0) + pygam.s(1)).gridsearch(x, y,lam=lams, progress=False)
    print('~'*30)
    print('~'*30)
    print('GAM results for: ', cover_type)
    print('~'*30)
    print('~'*20)
    single_gam.summary()
    for i, term in enumerate(single_gam.terms):
        if term.isintercept:
            continue

        XX = single_gam.generate_X_grid(term=i)
        pdep, confi = single_gam.partial_dependence(term=i, X=XX, width=0.95)

        plt.figure()
        plt.plot(XX[:, term.feature], pdep)
        plt.plot(XX[:, term.feature], confi, c='r', ls='--')
        plt.title(repr(term))
        plt.show()


# Simplest figure: Number and area of reservoirs over time in MT

In [ ]:
def read_process_basicstats(res_y):
    csv = './out/res_lulc_processed/res_lulc_{}.csv'.format(res_y)
    lulc_df = pd.read_csv(csv).set_index('id_in_tile')
    lulc_df['area_ha'] = lulc_df['area']*100/10000
    lulc_df = lulc_df.loc[lulc_df['area_ha']<100]
    out_dict = {
        'year': res_y,
        'count': lulc_df.shape[0],
        'area': lulc_df['area'].sum()
    } 
    return out_dict

all_dicts = [read_process_basicstats(res_y) for res_y in year_range]

In [ ]:
basic_stats_df = pd.DataFrame(all_dicts).set_index('year')
basic_stats_df['area_ha'] = basic_stats_df['area']*100/10000 # HA
basic_stats_df['area_km'] = basic_stats_df['area']*100/(1000*1000) # km2
basic_stats_df['count_k'] = basic_stats_df['count']/1000

In [ ]:
color_dict = {'ls5': '#66c2a5',
              'ls7': '#fc8d62',
              'ls8': '#8da0cb'}

In [ ]:
# Calculate which landsats for which years
ls_year_dict = {
    'ls5':np.concatenate([np.arange(1985, 2001), np.arange(2003, 2012)]),
    'ls7': np.array([2001, 2002, 2012, 2013]),
    'ls8': np.arange(2014, 2022)
}

ls_list = []
for res_y in year_range:
    for key, value in ls_year_dict.items():
        if res_y in value:
            ls_list.append(key)

basic_stats_df['satellite'] = ls_list
year_satellite = basic_stats_df.reset_index().set_index('year')[['satellite']]

In [ ]:

def get_ls_color(y):
    return color_dict[year_satellite.loc[y].values[0]]

def color_by_ls(df, column, ax):
    year_values = np.stack([basic_stats_df.index.get_level_values('year'),
                        basic_stats_df[column]],
                       axis=1)
    for start, stop in zip(year_values[:-1],year_values[1:]):
        x, y = zip(start, stop)
        ls_shortname = year_satellite.loc[x[0]].values[0]
        ls_name = 'Landsat {}'.format(ls_shortname[-1])
        if ls_name not in ax.get_legend_handles_labels()[1]:
            ax.plot(x, y, color=get_ls_color(x[0]), lw=2.5, label=ls_name,
                   solid_capstyle='round')
        else:
            ax.plot(x, y, color=get_ls_color(x[0]), lw=2.5,
                   solid_capstyle='round')

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(12,5))
color_by_ls(basic_stats_df, 'count_k', axs[0])
color_by_ls(basic_stats_df, 'area_km', axs[1])
axs[0].set_ylabel('Total Reservoir Count (thousands)')
axs[0].set_xlabel('Year')
axs[1].set_ylabel('Total Reservoir Surface Area ($km^2$)')
axs[1].set_xlabel('Year')
axs[0].legend()
fig.tight_layout()